In [4]:
import csv
import pymongo
from pymongo import MongoClient
import pandas as pd

In [9]:
"""
目標：利用我們的最終電影名單 Final.csv
到mongoDB的3個collections裡，去抓個部電影的資料，
然後整理成一個document，再回傳到monDB裡，
這樣我們每部電影的資料就集中，不會分散在不同的collections中
"""
# 將我們的最終電影名單讀進來，第一行是標題略過不讀，不給標題
data = pd.read_csv('Final.csv', encoding='utf-8', skiprows=[0], header=None)

#print len(data)

# 將讀進的來的dataframe，取各自電影的list名單，因為是dataframe，所以會有順序，它會從第一筆讀到最後一筆，產生list
douban = data[0].values.tolist()    # 豆瓣在第一column
truemovie = data[1].values.tolist()    # 觸電網在第二column
atmovie = data[2].values.tolist()    # 開眼在第三column

# 建立MongoDB連線
client = MongoClient("localhost", 27018)

#我們自己訂的db 和 collection
db = client.final
collect = db.threeInOne

# 利用迴圈將每筆資料讀出來，資料長度 = len(data)
i = 0
while i < len(data):
    
    # 用來裝每一部電影的空dict，之後要把它insert回去mongoDB
    movie = {}
    print douban[i]
    
    # 利用上面的名單，去找出我們的目標電影，因為當初的dataframe是有順序的，所以在同一個i值底下，就是同一部電影
    douban_cursor = db.douban.find({"movie_id": str(douban[i])})
    truemovie_cursor = db.truemovie.find({"url": str(truemovie[i])})
    atmovie_cursor = db.atmovie.find({"atmovie_id": str(atmovie[i])})
    
#    print douban_cursor

    """
    下面的目標：是要 1. 先從找到的電影中，拿到它的key值，並讓它變成新的document的key
                    2. 利用拿到的key值，就可以得到該key的value
                    就可以產生新的電影document
    """
    # 針對douban，因為find出來的cursor是一個object，所以要這樣把裡面的document讀出來，其實一個cursor也只有一個document
    for douban_document in douban_cursor:
        
        # 用來裝document裡面，每個欄位值的空list
        douban_field_name_list = []
        
        # 取出document中的所有key值
        for douban_key in douban_document:
            # 單一欄位，要有兩個值["要塞進mongoDB的新key", "舊的key(for取value用)"],每一個欄位要用新的list裝，所以放在迴圈裡
            douba_sigle_movie = []
            
            # 不是_id就通通留下來
            if douban_key != "_id":
                douba_sigle_movie.append("d_" + douban_key) # 產生新key
                douba_sigle_movie.append(douban_key) #舊key(for取值)
                
                # 將組合好的[新key, 舊key]放進douban_field_name_list
                douban_field_name_list.append(douba_sigle_movie)
        
        # 將douban_field_name_list裡的每個list→[新key, 舊key]讀出來，然後拿給movie.update()用，
        # update()裡面放{key:value}，key=douban_field_name[0], value=douban_document[douban_field_name[1]]
        [movie.update({douban_field_name[0] : douban_document[douban_field_name[1]]}) for douban_field_name in douban_field_name_list]

    for truemovie_document in truemovie_cursor:
        
        truemovie_field_name_list = []
        
        for truemovie_key in truemovie_document:
            truemovie_sigle_movie = []
            if truemovie_key != "_id":
                truemovie_sigle_movie.append("true_" + truemovie_key)
                truemovie_sigle_movie.append(truemovie_key)
                truemovie_field_name_list.append(truemovie_sigle_movie)
        
        [movie.update({truemovie_field_name[0] : truemovie_document[truemovie_field_name[1]]}) for truemovie_field_name in truemovie_field_name_list]

    for atmovie_document in atmovie_cursor:
        
        atmovie_field_name_list = []
        
        for atmovie_key in atmovie_document:
            atmovie_sigle_movie = []
            if atmovie_key != "_id":
                atmovie_sigle_movie.append("at_" + atmovie_key)
                atmovie_sigle_movie.append(atmovie_key)
                atmovie_field_name_list.append(atmovie_sigle_movie)
        
        [movie.update({atmovie_field_name[0] : atmovie_document[atmovie_field_name[1]]}) for atmovie_field_name in atmovie_field_name_list]
        
        print movie
        i += 1
        
        # 三部電影的放進movie{}，就完成了，把movie{} insert進去mongoDB就好了~
        collect.insert_one(movie)

4649
1890095
{u'd_movie_id': u'1890095', u'true_url': u'http://www.truemovie.com/2006moviedata/GrandMeaulnes.htm', u'at_title_t': u'\n\r\n\t\u7f8e\u9e97\u7684\u7d04\u5b9a Le Grand Meaulnes \n', u'at_poster': u'http://l10l010l3322l1.photos.atmovies.com.tw:8080/film/2006/G/fgfr50481349/poster/pl_fgfr50481349_0002.jpg', u'at_actor': [u'\u5c3c\u53e4\u62c9\u8fea\u4f5b\u4f11\u723e Nicolas Duvauchelle', u'\u5c1a\u5df4\u5824\u83ab\u91cc\u8036 Jean-Baptiste Maunier', u'\u514b\u857e\u66fc\u7d72\u6ce2\u897f Clemence Poesy', u'\u5c1a\u76ae\u723e\u99ac\u5229\u8036 Jean-Pierre Marielle', u'\u83f2\u529b\u6d66\u6258\u6d77\u9813 Philippe Torreton', u'\u611b\u871c\u9e97\u5fb7\u594e\u6069 Emilie Dequenne', u'\u99ac\u529b\u514b\u9f4a\u72c4 Malik Zidi', u'Valerie Stroh', u'\u4f5b\u863f\u502b\u65af\u6e6f\u746a\u68ee Florence Thomassin'], u'true_type': [u'\u5287\u60c5'], u'd_title1': u'', u'at_release_date': u'2007/06/29', u'at_rating': u'6.7', u'at_writer': [u'\u4e9e\u862d\u5085\u5c3c\u8449 Alain-Fournier',